Perform the following steps to set up an Azure Computer Vision service. In this chapter, we will use the [Azure CLI a lot to create and delete our resources](https://docs.microsoft.com/en-us/azure/cognitive-services/cognitive-services-apis-create-account-cli?tabs=linux). Cognitive Service you setup, then you might be asked to provision the service using the Azure Portal as you need to accept the Responsible AI terms. You will likely see this error message:
```
This subscription cannot create ComputerVision until you agree to Responsible AI terms for this resource. You can agree to Responsible AI terms by creating a resource through the Azure Portal then trying again. For more detail go to https://go.microsoft.com/fwlink/?linkid=2164911
```
First, perform a login according to step 1. As you may have more than one subscriptions in your directory, we set the standard subscription that Azure CLI will use in 2. Finally, there are individual setup and destroy scripts for each service.

1. `az login`
2. `az account set --subscription "<YOUR SUBSCRIPTION ID>"`
3. `./computervision-setup.sh`

In [ ]:
%%bash
./computervision-setup.sh

In [2]:
%load_ext dotenv
%dotenv
%load_ext pycodestyle_magic

# Cognitive Services Computer Vision

- [Azure-Samples Quickstart code](https://github.com/Azure-Samples/cognitive-services-quickstart-code)
- [Azure Cognitive Services Computer Vision SDK for Python](https://docs.microsoft.com/en-us/python/api/overview/azure/cognitiveservices-vision-computervision-readme?view=azure-python)

🟥 As Python is the de facto standard language in Machine Learning we will make use of it in most cases. To see other SDK in other languages, please refer to Azure-Samples Quickstart code repository (see above).

## OCR Text recognition

- [OCR Quickstart](https://docs.microsoft.com/en-us/azure/cognitive-services/computer-vision/quickstarts-sdk/client-library?tabs=visual-studio&pivots=programming-language-python)

In [ ]:
import time
import os
from msrest.authentication import CognitiveServicesCredentials
from azure.cognitiveservices.vision.computervision.models import OperationStatusCodes
from azure.cognitiveservices.vision.computervision import ComputerVisionClient

region = os.environ["ACCOUNT_REGION"]
key = os.environ["ACCOUNT_KEY"]

credentials = CognitiveServicesCredentials(key)
client = ComputerVisionClient(
    endpoint="https://" + region + ".api.cognitive.microsoft.com/",
    credentials=credentials
)

# set link if you like to use your own image
url = "https://github.com/Azure-Samples/cognitive-services-python-sdk-samples/raw/master/samples/vision/images/make_things_happen.jpg"
raw = True
numberOfCharsInOperationId = 36

# SDK call
rawHttpResponse = client.read(url, language="en", raw=True)

# Get ID from returned headers
operation_location = rawHttpResponse.headers["Operation-Location"]
id_location = len(operation_location) - numberOfCharsInOperationId
operation_id = operation_location[id_location:]

# Call the "GET" API and wait for it to retrieve the results
while True:
    result = client.get_read_result(operation_id)
    if result.status not in ['notStarted', 'running']:
        break
    time.sleep(1)

# Get data
if result.status == OperationStatusCodes.succeeded:

    for line in result.analyze_result.read_results[0].lines:
        print(line.text)
        print(line.bounding_box)


# Image Analysis

In this example we want to analyze an image by tags. We use the `analyze_image` method to perform image analyis and set the visual_features property accordingly to what features we would like to have our image analyzed.

### Content Tagging

In [ ]:
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import VisualFeatureTypes
from msrest.authentication import CognitiveServicesCredentials

import os
region = os.environ['ACCOUNT_REGION']
key = os.environ['ACCOUNT_KEY']

credentials = CognitiveServicesCredentials(key)
client = ComputerVisionClient(
    endpoint="https://" + region + ".api.cognitive.microsoft.com/",
    credentials=credentials
)

# set link if you like to use your own image
url = "https://images.pexels.com/photos/338515/pexels-photo-338515.jpeg"

image_analysis = client.analyze_image(
    url, visual_features=[VisualFeatureTypes.tags])

for tag in image_analysis.tags:
    print(tag)

### Domain-specific content

In [ ]:
domain = "landmarks"
url = "https://images.pexels.com/photos/1603650/pexels-photo-1603650.jpeg?auto=compress&cs=tinysrgb&h=750&w=1260"

analysis = client.analyze_image_by_domain(domain, url)

for landmark in analysis.result["landmarks"]:
    print(landmark["name"])
    print(landmark["confidence"])

### Creating a thumbnail

In [ ]:
from PIL import Image
import io

width = 50
height = 50
url = "https://images.pexels.com/photos/1603650/pexels-photo-1603650.jpeg?auto=compress&cs=tinysrgb&h=750&w=1260"

thumbnail = client.generate_thumbnail(width, height, url)

for x in thumbnail:
    image = Image.open(io.BytesIO(x))

# image.save('thumbnail.jpg')
image.show()

## Spatial Analysis

👉🏼 https://github.com/Azure-Samples/cognitive-services-spatial-analysis

In [ ]:
%%bash
./cogsrv-destroy.sh